# Conversational agent

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
from langchain.tools import tool

In [5]:
import requests
from pydantic import BaseModel, Field
import datetime

# Note This is the same function from last lesson
# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [6]:
import wikipedia

# Note This is the same function from last lesson
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [7]:
tools = [get_current_temperature, search_wikipedia]

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [9]:
functions = [convert_to_openai_function(f) for f in tools]

model = ChatOpenAI(temperature=0).bind(functions=functions)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [8]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [9]:
result.tool

'get_current_temperature'

In [10]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [10]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
    # These are action and observation pairs
])

In [11]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [17]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
    # This is required
})

In [18]:
result1.tool

'get_current_temperature'

In [19]:
observation = get_current_temperature.invoke(result1.tool_input)

In [20]:
observation

'The current temperature is 12.9°C'

In [21]:
type(result1)

langchain_core.agents.AgentActionMessageLog

In [12]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [23]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 136, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-37322d0a-3b68-4d0e-85a1-1a1964454205-0')]

In [24]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 136, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-37322d0a-3b68-4d0e-85a1-1a1964454205-0'),
 FunctionMessage(content='The current temperature is 12.9°C', name='get_current_temperature')]

In [25]:
result2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [26]:
result2
# This will contain final output? 

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 12.9°C.'}, log='The current temperature in San Francisco is 12.9°C.')

In [13]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    intermediate_steps = []

    while True:
        # This is different from previous lesson 
        # the invokation is moved inside the function, inside the while loop 
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })

        if isinstance(result, AgentFinish):
            # Note: This is the break condition
            return result
        
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }

        tool = tools[result.tool]

        observation = tool.invoke(result.tool_input)

        intermediate_steps.append((result, observation))

In [14]:
from langchain.schema.runnable import RunnablePassthrough

# Note: This is like object.assign in Typescript,
# Its just adding a new key named agent_scratchpad to the input dict?
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [15]:
def run_agent(user_input):
    intermediate_steps = []
    
    while True:
        # Note: This is a slightly more simplified loop than previous one
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })

        if isinstance(result, AgentFinish):
            return result
        
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }

        tool = tools[result.tool]
        
        observation = tool.run(result.tool_input)
        
        intermediate_steps.append((result, observation))

In [30]:
run_agent("What is the weather in sf?")

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 11.4°C.'}, log='The current temperature in San Francisco is 11.4°C.')

In [31]:
run_agent("What is Langchain?")

AgentFinish(return_values={'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework that can be used for document analysis and summarization, chatbots, and code analysis.'}, log='LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework that can be used for document analysis and summarization, chatbots, and code analysis.')

In [32]:
run_agent("Hi how are you")

AgentFinish(return_values={'output': "Hello! I'm here and ready to assist you. How can I help you today?"}, log="Hello! I'm here and ready to assist you. How can I help you today?")

In [16]:
from langchain.agents import AgentExecutor

# The above method was too tedious, but AgentExecutor simplifies it a lot
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [34]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: DataStax
Summary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.



Page: Sentence embedding
Summary: In natural language processing, a sentence embedding refe

{'input': 'what is langchain?',
 'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework that can be used for document analysis and summarization, chatbots, and code analysis.'}

In [35]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [37]:
agent_executor.invoke({"input": "what is my name"})
# Currently this doesnt have memory of previous messages



> Entering new AgentExecutor chain...
I'm sorry, I don't have access to your personal information. How can I assist you today?

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, I don't have access to your personal information. How can I assist you today?"}

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [18]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [19]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [20]:
# adding memory to the AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [42]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?')],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [43]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?')],
 'output': 'You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?'}

In [44]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 11.4°CThe current temperature in San Francisco is 11.4°C. Is there anything else you would like to know?

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The current temperature in San Francisco is 11.4°C. Is there anything else you would like to know?')],
 'output': 'The current temperature in San Francisco is 11.4°C. Is there anything else you would like to know?'}

In [21]:
class ChatBot:
    def __init__(self, tools):
        self.tools = tools
        self.functions = [convert_to_openai_function(f) for f in self.tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        # Note: This has some memory persistance issues I dont know why
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=True, memory=self.memory)

    def get_response(self, query):
        result = self.qa.invoke({"input": query})
        return result['output']

In [22]:
chatbot = ChatBot(tools)

In [23]:
user_input = "hi my name is Rutam"
response = chatbot.get_response(user_input)



> Entering new AgentExecutor chain...
Hello Rutam! How can I assist you today?

> Finished chain.


In [24]:
user_input = "why do people call donald trump orange?"
response = chatbot.get_response(user_input)



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Donald Trump'}`


Page: Donald Trump
Summary: Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021.
Trump received a Bachelor of Science in economics from the University of Pennsylvania in 1968. His father named him president of his real estate business in 1971. Trump renamed it the Trump Organization and reoriented the company toward building and renovating skyscrapers, hotels, casinos, and golf courses. After a series of business failures in the late twentieth century, he successfully launched side ventures, mostly licensing the Trump name. From 2004 to 2015, he co-produced and hosted the reality television series The Apprentice. He and his businesses have been plaintiff or defendant in more than 4,000 legal actions, including six business bankruptcies.
Trump won the 2016 presiden

In [25]:
user_input = "what is my name"
response = chatbot.get_response(user_input)



> Entering new AgentExecutor chain...
Your name is Rutam.

> Finished chain.
